In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from python.psfs import PsfDonut2D
from python.estimators import est_qLSQiter2D
from python.fluorophores import FlBlinkBleach
from python.simulators import Simulator

In [ ]:
fl = FlBlinkBleach()
sim = Simulator(fluorophores=fl)
sim.fluorophores.fast_toff = 0.1  # off-time ms
sim.fluorophores.fast_ton = 0.1  # on-time ms

In [ ]:
repetitions = 1  # how often to repeat the pattern scan
L = 75
pointdwelltimerep = 0.1  # ms, for all repetitions
pointdwelltime=pointdwelltimerep/repetitions  # ms
orbitpoints = 4
probecenter = True
psf_donut = PsfDonut2D()
sim.definePattern("donut", 
                  psf_donut, 
                  makepattern = "orbitscan", 
                  orbitpoints = orbitpoints,
                  probecenter = probecenter,
                  orbitL = L,
                  pointdwelltime = pointdwelltime,
                  laserpower = 100,
                  repetitions = repetitions)
sim.defineComponent("estsq", "estimator", est_qLSQiter2D, parameters=[L,probecenter], dim=(0,1))
out = sim.runSequence(["donut", "estsq"])
sim.summarize_results(out)

In [ ]:
## plot std vs repetitions
allrepetitions = np.arange(25)+1
stdx = np.zeros((len(allrepetitions),1))
stdy = np.zeros((len(allrepetitions),1))
stdxrel = np.zeros((len(allrepetitions),1))
stdyrel = np.zeros((len(allrepetitions),1))
biasx = np.zeros((len(allrepetitions),1))
biasy = np.zeros((len(allrepetitions),1))

for k in range(len(allrepetitions)):
    pointdwelltime=pointdwelltimerep/allrepetitions[k]  # us
    sim.definePattern("donut4", 
                      psf_donut, 
                      makepattern = "orbitscan", 
                      orbitpoints = orbitpoints,
                      probecenter = probecenter,
                      orbitL = L,
                      pointdwelltime = pointdwelltime,
                      laserpower = 100,
                      repetitions = allrepetitions[k])
    out = sim.runSequence(["donut4", "estsq"], maxlocs=3000)
    bright = out.loc.phot > np.quantile(out.loc.phot,0.1)  # filter out localizations that are too dim, 
                                                           # outliers from fluorophores that are mostly off
    converged = (np.abs(out.loc.xnm)<L) & (np.abs(out.loc.ynm)<L)  # estimator is artificially bound by +/-L. Here we 
                                                                   # remove localizations where the estimator converged to this limit.
    sr=sim.summarize_results(out, filter=bright&converged, display=False)
    stdx[k] = sr.std[0]
    stdy[k] = sr.std[1]
    stdxrel[k] = stdx[k]/sr.sCRB[0]
    stdyrel[k] = stdy[k]/sr.sCRB[1]
    biasx[k] = sr.bias[0]
    biasy[k] = sr.bias[1]


In [ ]:
sr.phot

In [ ]:
plt.figure()
plt.plot(allrepetitions,stdxrel,allrepetitions,stdyrel,allrepetitions,allrepetitions*0+1,'k')
plt.xlabel('repetitions')
plt.ylabel('std/sCRB')
plt.title("flickering fluorophores, mitigation by repetitive scanning")

## grid search

In [ ]:
pointdwelltimerep = 0.1  # ms
# pointdwelltimereps = [0.01, 0.03, 0.1]  # ms
t_ons = [0.01, 0.03, 0.1]  # ms
t_offs = [0.01, 0.03, 0.1]  # ms
photons = np.array([10, 100, 100000])*(100/411*100/41)

allrepetitions = np.arange(25)+1

M, P, Q, K = len(t_ons), len(t_offs), len(photons), len(allrepetitions)

## plot std vs repetitions
stdx = np.zeros((M,P,Q,K,1))
stdy = np.zeros((M,P,Q,K,1))
stdxrel = np.zeros((M,P,Q,K,1))
stdyrel = np.zeros((M,P,Q,K,1))
biasx = np.zeros((M,P,Q,K,1))
biasy = np.zeros((M,P,Q,K,1))
photonsx = np.zeros((M,P,Q,K,1))
crbx = np.zeros((M,P,Q,K,1))

for m, t_on in enumerate(t_ons):
    sim.fluorophores.fast_ton = t_on  # on-time ms
    for p, t_off in enumerate(t_offs):
        sim.fluorophores.fast_toff = t_off  # off-time ms
        for q, photon in enumerate(photons):
            sim.fluorophores.brightness = photon*((t_on+t_off)/t_on)
            for k in range(K):
                pointdwelltime=pointdwelltimerep/allrepetitions[k]  # us
                sim.definePattern("donut4", 
                                psf_donut, 
                                makepattern = "orbitscan", 
                                orbitpoints = orbitpoints,
                                probecenter = probecenter,
                                orbitL = L,
                                pointdwelltime = pointdwelltime,
                                laserpower = 100,
                                repetitions = allrepetitions[k])
                out = sim.runSequence(["donut4", "estsq"], maxlocs=3000)
                bright = out.loc.phot > np.quantile(out.loc.phot,0.1)  # filter out localizations that are too dim, 
                                                                    # outliers from fluorophores that are mostly off
                converged = (np.abs(out.loc.xnm)<L) & (np.abs(out.loc.ynm)<L)  # estimator is artificially bound by +/-L. Here we 
                                                                            # remove localizations where the estimator converged to this limit.
                sr=sim.summarize_results(out, filter=bright&converged, display=False)
                stdx[m,p,q,k] = sr.std[0]
                stdy[m,p,q,k] = sr.std[1]
                stdxrel[m,p,q,k] = stdx[m,p,q,k]/sr.sCRB[0]
                stdyrel[m,p,q,k] = stdy[m,p,q,k]/sr.sCRB[1]
                biasx[m,p,q,k] = sr.bias[0]
                biasy[m,p,q,k] = sr.bias[1]
                photonsx[m,p,q,k] = sr.phot
                crbx[m,p,q,k] = sr.sCRB[0]


In [ ]:
# plot fixed ton, pointdwelltime vs toff
tonidx = -1
pointdwelltimeidx = -1

fig, axs = plt.subplots(4,3, figsize=(10,15))
for tonidx in range(M):
    for toffidx in range(P):
        ax = axs[tonidx, toffidx]
        ax.plot(allrepetitions,
                stdx[tonidx, toffidx, 1], 'ro', label=f"N={int(photonsx[tonidx,toffidx,1].mean()):d}")
        ax.plot(allrepetitions, crbx[tonidx, toffidx, 1],'r--', label=None)
        ax.plot(allrepetitions, stdx[tonidx, toffidx, 2],'k', label=None)
        ax.plot(allrepetitions, np.sqrt(stdx[tonidx, toffidx, 2]**2+crbx[tonidx, toffidx, 1]**2),'r', label=None)
        ax.set_xlabel('repetitions')
        ax.set_ylabel('x: STD (nm)')
        ax.set_ylim([0,12])
        ax.set_xlim([0,25])
        ax.set_title("$t_\\text{on}$ = " + f"{t_ons[tonidx]:.2f} ms " + "$t_\\text{off}$ = " + f"{t_offs[toffidx]:.2f} ms")
        # plt.legend()
# fig.tight_layout()
# fig.savefig("flickering.pdf")

## Eilers et al., PNAS (2018)

In [ ]:
tcpdwelltimee = 0.4  # ms
t_one = 0.057    # ms
t_offe = 0.0057  # ms

## plot std vs repetitions
allrepetitionse = np.arange(25)+1

photonse = np.array([100, 100000])*(100/411)
Qe, Ke = len(photonse), len(allrepetitionse)

stdxe = np.zeros((Qe,Ke,1))
stdye = np.zeros((Qe,Ke,1))
stdxrele = np.zeros((Qe,Ke,1))
stdyrele = np.zeros((Qe,Ke,1))
biasxe = np.zeros((Qe,Ke,1))
biasye = np.zeros((Qe,Ke,1))
photonsxe = np.zeros((Qe,Ke,1))
crbxe = np.zeros((Qe,Ke,1))

sim.fluorophores.fast_ton = t_one  # on-time ms
sim.fluorophores.fast_toff = t_offe  # off-time ms

for q in range(Qe):
    sim.fluorophores.brightness = photonse[q]*((t_on+t_off)/t_on)    
    for k in range(Ke):
        pointdwelltime=tcpdwelltimee/orbitpoints/allrepetitionse[k]  # us
        sim.definePattern("donut4", 
                        psf_donut, 
                        makepattern = "orbitscan", 
                        orbitpoints = orbitpoints,
                        probecenter = probecenter,
                        orbitL = L,
                        pointdwelltime = pointdwelltime,
                        laserpower = 100,
                        repetitions = allrepetitionse[k])
        out = sim.runSequence(["donut4", "estsq"], maxlocs=3000)
        bright = out.loc.phot > np.quantile(out.loc.phot,0.1)  # filter out localizations that are too dim, 
                                                            # outliers from fluorophores that are mostly off
        converged = (np.abs(out.loc.xnm)<L) & (np.abs(out.loc.ynm)<L)  # estimator is artificially bound by +/-L. Here we 
                                                                    # remove localizations where the estimator converged to this limit.
        sr=sim.summarize_results(out, filter=bright&converged, display=False)
        stdxe[q,k] = sr.std[0]
        stdye[q,k] = sr.std[1]
        stdxrele[q,k] = stdxe[q,k]/sr.sCRB[0]
        stdyrele[q,k] = stdye[q,k]/sr.sCRB[1]
        biasxe[q,k] = sr.bias[0]
        biasye[q,k] = sr.bias[1]
        photonsxe[q,k] = sr.phot
        crbxe[q,k] = sr.sCRB[0]

In [ ]:
# plot fixed ton, pointdwelltime vs toff
tonidx = -1
pointdwelltimeidx = -1

# fig, ax = plt.subplots(1,1, figsize=(10,10))
ax = axs[3,0]
ax.plot(allrepetitions,stdxe[0], 'ro')
ax.plot(allrepetitions, crbxe[0],'r--', label=None)
ax.plot(allrepetitions, stdxe[1],'k', label=None)
ax.plot(allrepetitions, np.sqrt(stdxe[1]**2+crbxe[0]**2),'r', label=None)
# ax.plot(allrepetitions, allrepetitions*0+1,'k', label=None)
ax.set_xlabel('repetitions')
ax.set_ylabel('x: STD (nm)')
ax.set_ylim([0,12])
ax.set_xlim([0,25])
ax.set_title("Eilers et al. 2018\n $t_\\text{on}$ = " + f"{t_one:.2f} ms " + "$t_\\text{off}$ = " + f"{t_offe:.2f} ms")
# ax.set_title("\n $t_\\text{on}$ = " + str(t_on) + f"ms toff = {t_off} ms")
# plt.legend()
axs[3,1].set_axis_off()
axs[3,2].set_axis_off()
fig.tight_layout()
fig.savefig("flickering_eilers.pdf")